In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

2026-02-13 14:59:37.978137: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-13 14:59:38.423367: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-13 14:59:40.364570: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
# Carregar dados
base = pd.read_csv('/home/Ront/Documentos/Card-14/Pratica/Seção 6/autos.csv', encoding='ISO-8859-1')
base.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
# Remova as colunas que não ajudam no preço
base = base.drop(['dateCrawled', 'nrOfPictures', 'lastSeen', 'postalCode', 'dateCreated', 'name', 'seller', 'offerType'], axis=1)
base = base.dropna(subset=['vehicleType', 'gearbox', 'model', 'fuelType', 'notRepairedDamage'])

In [5]:
#Filtro de preços para evitar "ruído" no aprendizado
base = base[base.price > 100]
base = base[base.price < 350000]

In [6]:
#Separação X são os previsores y é o preço
X = base.iloc[:, 1:12].values 
y = base.iloc[:, 0].values.astype('float32') # Já converte o preço para número real

In [7]:
onehotencoder = ColumnTransformer(
    transformers=[('OneHot', OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])],
    remainder='passthrough'
)
X = onehotencoder.fit_transform(X).toarray().astype('float32')

In [8]:
# Criar modelo de regressão
regressor = Sequential([
    tf.keras.layers.InputLayer(shape=(X.shape[1],)),
    tf.keras.layers.Dense(units=158, activation='relu'),
    tf.keras.layers.Dense(units=158, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='linear')
])

2026-02-13 15:00:27.430470: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
# Compilar com Adam
regressor.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])

In [10]:
# Treinar o modelo
regressor.fit(X, y, batch_size=300, epochs=50)

Epoch 1/50


2026-02-13 15:00:46.322596: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 322249408 exceeds 10% of free system memory.


856/856 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4228.2671 - mean_absolute_error: 4228.2676
Epoch 2/50
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3603.0322 - mean_absolute_error: 3603.0322
Epoch 3/50
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 3402.8984 - mean_absolute_error: 3402.8984
Epoch 4/50
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 3261.2310 - mean_absolute_error: 3261.2310
Epoch 5/50
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3152.3645 - mean_absolute_error: 3152.3645
Epoch 6/50
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3037.2314 - mean_absolute_error: 3037.2314
Epoch 7/50
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2972.7295 - mean_absolute_error: 2972.7295
Epoch 8/50
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2935.6165 - mean_absolute_error: 2935.6165
Epoch 9/50
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2894.1594 - mean_absolute_error: 2894.1594
Epoch 10/50
856/856 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2849.1130 - mean_absolu

In [11]:
# Fazer previsões
previsoes = regressor.predict(X)

2026-02-13 15:02:40.076363: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 322249408 exceeds 10% of free system memory.


8018/8018 ━━━━━━━━━━━━━━━━━━━━ 5s 642us/step


In [13]:
# Comparar médias
y.mean()

np.float32(6792.736)

In [14]:
previsoes.mean()

np.float32(6404.688)

In [15]:
# Visualizar algumas previsões
import numpy as np

comparacao = pd.DataFrame({
    'Real': y[:10],
    'Previsto': previsoes[:10].flatten()
})
comparacao

,Real,Previsto
0,1500.0,1522.302246
1,3600.0,6806.870117
2,650.0,1065.413086
3,2200.0,1915.374268
4,2000.0,1428.415771
5,2799.0,2809.353760
6,17999.0,20986.861328
7,1750.0,1123.235718
8,7550.0,4779.129395
9,1850.0,2114.884033
